# Fase de **Inferencia** | Verne Academy **Kaggle Competition**
### Por **Mario Jurado Galán**
Este notebook incluye:  
+ Carga de datos
+ Carga de objetos
+ Transformaciones
+ Predicción de resultados
+ Escritura en CSV

# 0. Librerias

In [ ]:
# ----Tratamiento de datos---
import functions as func
import pandas as pd

# ----Puesta en producción----
import pickle

# ----Warnings----
import warnings
warnings.filterwarnings('once')  

# 1. Carga de datos


In [ ]:
inference_df=pd.read_csv("../Datasets/MasterBI_Test.csv")

C:\Users\SumeriO\AppData\Local\Temp\ipykernel_32124\3889870005.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  inference_df=pd.read_csv("../Datasets/MasterBI_Test.csv")


# 2. Carga de objetos

In [ ]:

# Modelo
with open('output/model.pkl', 'rb') as c:
    model=pickle.load(c)
    
# Objetos de transformación

with open('output/numeric_imputer.pkl', 'rb') as c:
    numeric_imp=pickle.load(c)
    
with open('output/category_imputer.pkl', 'rb') as c:
    category_imp=pickle.load(c)
    
with open('output/onehot_encoder.pkl', 'rb') as c:
    onehot_enc=pickle.load(c)

with open('output/target_encoder.pkl', 'rb') as c:
    target_enc=pickle.load(c)

# Columnas
with open('output/columns.pkl', 'rb') as c:
    columns=pickle.load(c)

with open('output/delete_features.pkl','rb') as c:
    delete_features=pickle.load(c)

with open('output/synth_num_cols.pkl', 'rb') as c:
    synth_num_cols=pickle.load(c)

with open('output/synth_cat_cols.pkl', 'rb') as c:
    synth_cat_cols=pickle.load(c) 

with open('output/cat_cols.pkl', 'rb') as c:
    category_cols=pickle.load(c)

with open('output/num_cols.pkl', 'rb') as c:
    numeric_cols=pickle.load(c)

with open('output/onehot_cols.pkl', 'rb') as c:
    onehot_cols=pickle.load(c)

with open('output/target_cols.pkl', 'rb') as c:
    target_cols=pickle.load(c)

# 3. Transformaciones
+ Eliminación manual de columnas
+ Transformación manual de valores
+ Creacion de variables de máscara y contextuales
+ Imputación de nulos
+ Aplicación de encoders
+ Filtrado para quedarse con columnas resultantes de RFECV de la puesta en producción

In [ ]:
# Identificación de id y label
id="MachineIdentifier"
label="HasDetections"

In [ ]:
# Se eliminan las columnas borradas en la puesta en producción
inference_df=inference_df[[col for col in inference_df.columns if col not in delete_features]]

In [ ]:
# Se rellenan los nulos con valor 1 
inference_df["Census_IsWIMBootEnabled"]=inference_df["Census_IsWIMBootEnabled"].fillna(1.0)

In [ ]:
# Creación de variables de máscara
inference_df=func.add_mask_features(inference_df)

In [ ]:
# Creación de variables contextuales con las columnas provenientes de la puesta en producción
inference_df, added_cols=func.generate_grouped_stats(inference_df, synth_num_cols, synth_cat_cols)
inference_df=func.generate_synthetic_features(inference_df, added_cols)

c:\Users\SumeriO\Desktop\Master BI IA\TFM IA\Fase 2\functions.py:31: FutureWarning: The provided callable <function mean at 0x00000241BC710860> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_grouped = group_by_feat[num_feat].agg([np.mean, np.max, np.min]).reset_index()
c:\Users\SumeriO\Desktop\Master BI IA\TFM IA\Fase 2\functions.py:31: FutureWarning: The provided callable <function max at 0x00000241BC6FFE20> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_grouped = group_by_feat[num_feat].agg([np.mean, np.max, np.min]).reset_index()
c:\Users\SumeriO\Desktop\Master BI IA\TFM IA\Fase 2\functions.py:31: FutureWarning: The provided callable <function min at 0x00000241BC6FFF60> is currently using SeriesGroupBy.min. In a future version of

In [ ]:
# Se coloca el id como index del DataFrame
inference_df=inference_df.set_index(id)

### Imputación de nulos

In [ ]:
# Variables numéricas por la media
inference_df[numeric_cols]=numeric_imp.transform(inference_df[numeric_cols])

# Variables categóricas por el valor mas frecuente
inference_df[category_cols]=category_imp.transform(inference_df[category_cols])

### Category Encoders

In [ ]:
# Onehot Encoder
onehot_encoded=onehot_enc.transform(inference_df[onehot_cols])
inference_onehot_df=pd.DataFrame(data=onehot_encoded.toarray(), columns=onehot_enc.get_feature_names_out(onehot_cols), index=inference_df.index)

#Target Encoder
inference_target_df=target_enc.transform(inference_df[target_cols].astype("category"))

In [ ]:
# Fusión de los DataFrames resultantes de los encoders 
final_inference_df=inference_onehot_df.join(inference_target_df)

In [ ]:
# Comprobación de filas, columnas y nulos
print(f"--Dataset--\nRows:{final_inference_df.shape[0]}\nColumns:{final_inference_df.shape[1]}\n")
print(f"Cantidad de nulos: \nTrain: {final_inference_df.isna().sum().sum()}")

--Dataset--
Rows:321173
Columns:162

Cantidad de nulos: 
Train: 0


In [ ]:
# Se queda solo con las columnas resultantes del RFECV de la puuesta en producción
final_inference_df=final_inference_df[[col for col in final_inference_df.columns if col in columns]]

# 4. Predicción de resultados

In [ ]:
# Se generan las predicciones de los datos con el modelo previamente entrenado
predictions=model.predict(final_inference_df)

[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.8500000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8500000000000001


In [ ]:
# Se crea un DataFrame con el identificador MachineIdentifier y su correspondiente resultado en la columna HasDetections
results = pd.DataFrame()
results[id] = final_inference_df.reset_index()[id]
results[label] = predictions

In [ ]:
# Visualización del resultado
results.head()

,MachineIdentifier,HasDetections
0,961e21282c5c1474ce2330d8d528be6d,1
1,aa943fbc7cb6b6a77bdf7f61a1ba310c,0
2,917cad1527312b93d21d95cf35355be7,1
3,394f0ad0dbbc63d71940f11086388619,0
4,6e31f1fc89ac8cf6a2f59dad1fc5f643,1


In [ ]:
# Comprobar que no tiene nulos
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321173 entries, 0 to 321172
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   MachineIdentifier  321173 non-null  object
 1   HasDetections      321173 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.9+ MB


In [ ]:
# Comprobar distribución de los resultados
results[label].value_counts()

HasDetections
1    162995
0    158178
Name: count, dtype: int64

# 5. Escritura en CSV

In [ ]:
# Guardar los resultados como CSV para entregar en la competición de Kaggle
results.to_csv("Kaggle_Results_2_Fase2_Mario_Jurado.csv", index=False)